In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\Barytes\Desktop\NBS_project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 42
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 1, 'l': 0.8, 'cn': 0.039963209507789, 'Fn': 4.901428612819832, 'kn': 0.6659969709057025, 'omega_n': 1, 'Rn': 9.270270025833746}
{'s': 1, 'l': 0.8, 'cn': 0.02247956162689621, 'Fn': 3.116167224336399, 'kn': 0.7330880728874676, 'omega_n': 1, 'Rn': 7.653352487012544}
{'s': 1, 'l': 0.8, 'cn': 0.011646759543664197, 'Fn': 4.939819704323988, 'kn': 0.7162213204002108, 'omega_n': 1, 'Rn': 6.796707114575109}
{'s': 1, 'l': 0.8, 'cn': 0.024672360788274707, 'Fn': 3.6084844859190754, 'kn': 0.5623782158161189, 'omega_n': 1, 'Rn': 7.979027325089509}
{'s': 1, 'l': 0.8, 'cn': 0.05894823157779036, 'Fn': 3.278987721304084, 'kn': 0.44607232426760907, 'omega_n': 1, 'Rn': 7.104438488824009}
{'lambda0': 20, 'D0': 4, 'theta': 5, 'o': 10, 'omega_0': 2}


In [3]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
# r_swm = NBS.negotiation(esp, MDs,lam_swm, p_swm, Dmax_swm)
# print(f"SWM r: {r_swm}")

ValueError: Whenever the gradient is estimated via finite-differences, we require the Hessian to be estimated using one of the quasi-Newton strategies.

In [5]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -18.75182315629822
            Iterations: 47
            Function evaluations: 848
            Gradient evaluations: 47
Optimal λ: [4.97907581 3.00883218 5.29678281 3.60424262 3.11106658]
Optimal p: [4.4761472  2.90697151 4.74252994 3.38163942 2.99193267]
Optimal r: [15.06906356 14.12522029 15.04154758 13.85307223 13.60241808]
Optimal Dmax: 1.7309632931323466


In [5]:
s, l = MDs[0].s,  MDs[0].l
print("Baseline Opt:", p_opt/(s*l)-lam_opt-1e-6)
print("Baseline SWM:", p_swm/(s*l)-lam_swm-1e-6)

Baseline Opt: [-5.42415078 -7.15379174 -5.38695561 -6.6768145  -6.99607301]
Baseline SWM: [-5.42415545 -7.15379918 -5.38696038 -6.67681946 -6.99605116]


In [6]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")

 iter 0 ||lam-lam_hat||= 0.4017458325424779   alpha= [2.75906800e-08 2.76053349e-08 2.76058056e-08 2.75907510e-08
 8.12935653e-01]   beta= [3.98968427e-08 3.98968427e-08 3.98968427e-08 3.98968427e-08
 3.98968427e-08]
 iter 1 ||lam-lam_hat||= 0.012762244725815958   alpha= [2.18933351e-08 3.12659356e-08 9.08693032e-09 2.20961507e-08
 7.81030041e-01]   beta= [1.06242856e-08 1.06243377e-08 1.06243178e-08 1.06242964e-08
 1.06243083e-08]
SLSQP failed: Iteration limit exceeded
 iter 2 ||lam-lam_hat||= 0.02889519048050085   alpha= [ 7.34017869e-09 -3.44488513e-08 -9.09614695e-09 -3.39547799e-08
  7.44911053e-01]   beta= [-1.84646387e-09 -1.84638477e-09 -1.84640128e-09 -1.84644403e-09
 -1.84643875e-09]
 iter 3 ||lam-lam_hat||= 0.10378760168118771   alpha= [ 6.94406554e-09 -3.21968208e-09  8.61470695e-09  7.27074179e-10
  6.80043802e-01]   beta= [3.93314284e-10 3.93406710e-10 3.93392034e-10 3.93338605e-10
 3.93340478e-10]
 iter 4 ||lam-lam_hat||= 0.09694300202399082   alpha= [1.49422288e-08 2.43